## Simulator (single uav)

This notebook serves to get familiar with the simulator

In [1]:
from config import Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)
from simulator.gazebo.config import GazWP
from simulator.QGroundControl.config import QGCWP

clean()

## Simulation Configuration

1. **Set the simulation origin in GRA (Global Geodetic Reference) coordinates**  
   * The ENU (East-North-Up) origin is used only for the Gazebo representation. Typically, the ENU origin is taken as $(x=0, y=0)$.

2. **Set UAV base home and path**  
   * **Base home**: The starting UAV position (relative to the simulation origin). It is given in ENU coordinates for easier use.  
   * **Base path**: The UAV trajectory (relative to the uav base home). It is also given in ENU coordinates for easier use.

3. **UAV color**  
   * **Color**: Pick a color for the drone. This color is used for both the drone and its waypoints in Gazebo. For more complex setups with multiple UAVs and GCSs (Ground Control Stations), colors are typically used to distinguish which GCS or provider each UAV belongs to.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_home = ENUPose(x=5, y=5, z=0, heading=45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Visualizer

We can choose between three visualization modes:
* Gazebo  
* QGroundControl  
* No visualization  

Below we describe how to configure each of them for completeness.

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin= enu_origin,
                        world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)


gaz= Gazebo(gaz_config,gra_origin)

origin_gaz = GazWP(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

gaz.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc=QGC(qgc_config)

origin_qgc = QGCWP(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)

qgc.markers.append(origin_qgc)
qgc.show()

## No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
novis_config.add(base_home=base_home)

novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported). 
* **gcs_system_ids:** A dictionary where the keys are Ground Control Station (GCS) names and the values are the IDs of the UAVs assigned to each GCS.
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [6]:
qgc_config.vehicles[0].mission

Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.36328293355812), lon=np.float64(149.16516908417492), alt=np.float64(5.000003924369988)), color=blue), QGCWP(name='wp_1', pos=GRA(lat=np.float64(-35.36334666705318), lon=np.float64(149.16509127994456), alt=np.float64(5.000011747427456)), color=blue), QGCWP(name='wp_2', pos=GRA(lat=np.float64(-35.36328293338712), lon=np.float64(149.165013475923), alt=np.float64(5.000030659438993)), color=blue), QGCWP(name='wp_3', pos=GRA(lat=np.float64(-35.363219199941476), lon=np.float64(149.16509128015335), alt=np.float64(5.000022872010703)), color=blue), QGCWP(name='wp_4', pos=GRA(lat=np.float64(-35.36328293355812), lon=np.float64(149.16516908417492), alt=np.float64(5.000003924369988)), color=blue)], delay=0, land=True, speed=5.0)

In [7]:
simulator = Simulator(
    gra_origin=gra_origin,
	visualizers=[novis],
	gcs_system_ids={'simple':[1]},
	missions=[qgc_config.vehicles[0].mission],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

15:56:47 - Oracle ⚪ - INFO - 🙈 Running without visualization.
15:56:47 - Oracle ⚪ - INFO - 🚀 GCS simple launched (PID 1900989)


## Oracle checking

In [8]:
orac.run()

15:56:47 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 1 vehicles and 1 GCSs
15:58:12 - Oracle ⚪ - INFO - Received DONE from GCS simple
15:58:12 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
15:58:13 - Oracle ⚪ - INFO - ✅ All GCS threads completed
15:58:13 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
